In [ ]:
import Pkg # Importa el administrador de paquetes Pkg
Pkg.add("JuMP") # Instala el paquete JuMP

In [ ]:
Pkg.add("GLPK") # Instala el paquete GLPK

In [ ]:
Pkg.build("GLPK")

In [87]:
import Pkg # Importa el administrador de paquetes Pkg
Pkg.add("RandomNumbers")

  Updating registry at `C:\Users\Chunzen\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
Fetching: [========================================>]  100.0 %.0 %Fetching: [===>                                     ]  5.5 %Fetching: [====>                                    ]  8.1 %Fetching: [======>                                  ]  13.6 %                                ]  19.8 %Fetching: [=========>                               ]  21.1 %23.6 %.2 %8 %>                           ]  30.6 %>                          ]  33.2 %Fetching: [===============>                         ]  36.0 % %Fetching: [=================>                       ]  40.9 %]  42.5 %Fetching: [==================>                      ]  44.5 %6 %Fetching: [====================>                    ]  48.7 %Fetching: [=====================>                   ]  50.6 % %Fetching: [=======================>                 ]  56.4 %]  59.6 %]  62.3 %============================

$$
x_i = \begin{cases}
      1 : \text{Se extrae el bloque i}\\
      0 : \text{No se extrae el bloque i}\\
      \end{cases} \text{, con i = 1,...,18}\\
maximizar \:  \sum\limits_{i=1}^{18} (valor_i - costo_i)x_i \\
s.a. \:  x_a \leq \sum x_b \\
x_a \in \text{Bloque extraido}\\
b \in \text{Conjunto de precedencia para el bloque extraido}\\
$$

In [ ]:
using RandomNumbers.MersenneTwisters

prec = open("newman1.prec")
upit = open("newman1.upit")

rng = MT19937(123)
print(rand(rng, -10:10))

open("newman1.prec") do f
    line = 1
    while !eof(f)
        x = readline(f)
        
        list = split(x)
        #println("Numero ", list[1], ":", line[2])
        println("Numero ", list[2])
        
        n = size(list, 1)
        println("largo ", n)
        

        for j in 3:(parse(Int64, list[2])+2)
            print(list[j])
        end
        println()
        println()
        
        line += 1
    end
end


a = read(prec)


close(prec)
close(upit)

In [ ]:
using JuMP
using GLPK
using RandomNumbers.MersenneTwisters

# CONSTRUCCIÓN DEL MODELO
modelo = Model(with_optimizer(GLPK.Optimizer))

prec = open("newman1.prec")
blocks = open("newman1.blocks")
upit = open("newman1.upit")


c = []

open("newman1.upit") do f
    line = 1
    while !eof(f)
        a = readline(f)
        if(line >= 5)
            list = split(a)            
            append!(c, parse(Float64, list[2]))            
        end
        line += 1
    end
end

#rng = MersenneTwister(1234);
#rng = MT19937(123)

print(rand(rng, -10:10))

open("newman1.blocks") do f
    while !eof(f)
        a = readline(f)        
        list = split(a)
        #append!(c, parse(Float64, list[10]))            
        #append!(c, rand(rng, -10:10))          
    end
end

# VARIABLES DE DECISIÓN
@variable(modelo, x[1:1060], Bin) # Variable por cada bloque

# FUNCIÓN OBJETIVO
@objective(modelo, Max, sum(c[i]*x[i] for i in 1:1060))

# RESTRICCIONES

# Presedencia
open("newman1.prec") do f
    while !eof(f)
        a = readline(f)        
        list = split(a)
        n = size(list, 1)
        if n != 2 
            @constraint(modelo, parse(Int64, list[2]) * x[parse(Int64, list[1]) + 1] <= sum(x[parse(Int64, list[j]) + 1] for j in 3:(parse(Int64, list[2]) + 2)))
        end        
    end
end

#@constraint(modelo, 3*x[18] <= sum(x[j] for j in 12:14))



optimize!(modelo)
objective_value(modelo)

print("Valor objetivo: ", objective_value(modelo), ". \n\n") # Entrega el valor objetivo óptimo

print("Valor de las variables\n")
for i in 1:1060
    print("Valor x", i, ": ", value(x[i]), "\n")
end

close(prec)
close(blocks)
close(upit)